In [1]:
!pip install streamlit

  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=1e90ed6e495a6e3240a2d41e2fc80abcc11629cdb4a47350fe57463a87c18872
  Stored in directory: c:\users\soham\appdata\local\pip\cache\wheels\19\09\72\3eb74d236bb48bd0f3c6c3c83e4e0c5bbfcbcad7c6c3539db8
Successfully built validators


In [2]:
!pip install tensorflow


  Using cached google_auth-1.35.0-py2.py3-none-any.whl (152 kB)
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.6.0
    Uninstalling google-auth-2.6.0:
      Successfully uninstalled google-auth-2.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0


In [4]:
import tensorflow
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def image_gen_w_aug(train_parent_directory, test_parent_directory):
    
    train_datagen = ImageDataGenerator(rescale=1/255,
                                      rotation_range = 30,  
                                      zoom_range = 0.2, 
                                      width_shift_range=0.1,  
                                      height_shift_range=0.1,
                                      validation_split = 0.15)
    
  
    
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                       target_size = (75,75),
                                                       batch_size = 214,
                                                       class_mode = 'categorical',
                                                       subset='training')
    
    val_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                          target_size = (75,75),
                                                          batch_size = 37,
                                                          class_mode = 'categorical',
                                                          subset = 'validation')
    
    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                                     target_size=(75,75),
                                                     batch_size = 37,
                                                     class_mode = 'categorical')
    
    return train_generator, val_generator, test_generator


def model_output_for_TL (pre_trained_model, last_output):

    x = Flatten()(last_output)
    
    # Dense hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output neuron. 
    x = Dense(10, activation='softmax')(x)
    
    model = Model(pre_trained_model.input, x)
    
    return model


train_dir = os.path.join('D:\archive\data\food-101-tiny\train')
test_dir = os.path.join('D:\archive\data\food-101-tiny\valid')

train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, test_dir)

pre_trained_model = InceptionV3(input_shape = (75, 75, 10), 
                                include_top = False, 
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed3')
last_output = last_layer.output

model_TL = model_output_for_TL(pre_trained_model, last_output)
model_TL.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_TL = model_TL.fit(
      train_generator,
      steps_per_epoch=10,  
      epochs=20,
      verbose=1,
      validation_data = validation_generator)

tf.keras.models.save_model(model_TL,'my_model.hdf5')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
model = tf.keras.models.load_model('my_model.hdf5')

In [ ]:
import streamlit as st
st.write("""
         # Food item image Prediction
         """
         )
st.write("This is a image classification web app to predict food items")
file = st.file_uploader("Please upload an image file", type=["jpg", "png"])

In [ ]:
import cv2
from PIL import Image, ImageOps
import numpy as np
def import_and_predict(image_data, model):
    
        size = (150,150)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.
        
        img_reshape = img_resize[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        
        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    prediction = import_and_predict(image, model)
    
    if np.argmax(prediction) == 0:
        st.write("It is a apple pie!")
    elif np.argmax(prediction) == 1:
        st.write("It is a bibimbap!")
    elif np.argmax(prediction) == 2:
        st.write("It is a cannoli!")
    elif np.argmax(prediction) == 3:
        st.write("It is a edamame!")
    elif np.argmax(prediction) == 4:
        st.write("It is a falafel!")
    elif np.argmax(prediction) == 5:
        st.write("It is a french toast!")
    elif np.argmax(prediction) == 6:
        st.write("It is a ice cream!")
    elif np.argmax(prediction) == 7:
        st.write("It is a ramen!")
    elif np.argmax(prediction) == 8:
        st.write("It is a sushi!")
    else:
        st.write("It is a tiramisu!")
    
    st.text("Probability (0: apple pie, 1: bibimbap, 2: cannoli, 3: edamame, 4: falafel, 5: toast, 6: ice cream, 7: ramen, 8: sushi, 9: tiramisu")
    st.write(prediction)

In [ ]:
streamlit run food_app.py

In [ ]:
mkdir -p ~/.streamlit/                                               echo "\                       
[server]\n\                       
port = $PORT\n\                       
enableCORS = false\n\                       
headless = true\n\                       
\n\                       
" > ~/.streamlit/config.toml

In [ ]:
web: sh setup.sh && streamlit run food_app.py